In [146]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.analytical_models import SimpleFitCurve, ComplexFitCurve
from models.concurrency.xgboost import XGBoostPredictor
np.set_printoptions(suppress=True)

In [164]:
folder_name = "mixed_postgres"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=60)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)
#isolated_trace_df = pd.read_csv(f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/repeating_olap_batch_warmup.csv")
#isolated_trace_df = pd.read_csv(f"/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_redshift/repeating_olap_batch_warmup.csv")
isolated_trace_df["runtime"] = isolated_trace_df["run_time_s"]
isolated_rt_cache = dict()
for i, rows in isolated_trace_df.groupby("query_idx"):
    isolated_rt_cache[i] = np.median(rows["runtime"])

client 5 trace not found
client 6 trace not found
client 7 trace not found


In [165]:
for i, rows in concurrency_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), isolated_rt_cache[i], np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))

0 47 44.55030298233032 0.08360435607585498 0.0648715496063232 0.1127221584320068 0.009064831937329078
1 46 0.2543864250183105 0.009602893953737953 0.0061287879943847 0.0118489265441894 0.0012380041331746198
2 46 23.157404899597168 0.9576464528622834 0.933161735534668 1.0725383758544922 0.02632717798547574
3 48 0.283794641494751 0.00619843105475103 0.0048193931579589 0.0139329433441162 0.0016433393393040646
4 48 4.330385208129883 0.9231568972269694 0.8745567798614502 1.0311920642852783 0.03128915885380195
5 46 2.5214909315109253 0.13935753055240793 0.1248989105224609 0.1683840751647949 0.009970224752439706
6 48 2.5594241619110107 0.008190502723058013 0.005491018295288 0.0243732929229736 0.0036960411654215816
7 46 2.2192370891571045 0.07491088950115699 0.0629255771636962 0.0985400676727294 0.00732842676709004
8 46 0.3999779224395752 0.0017193607662034567 0.0009036064147949 0.0031909942626953 0.00044823678387808394
9 47 0.7344051599502563 0.002249961203717119 0.0014586448669433 0.00409960

In [145]:
np.random.seed(4)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0]

In [161]:
import numpy as np
import scipy.optimize as optimization
from models.concurrency.base_model import ConcurPredictor
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb


def simple_queueing_func(x, a1, a2, b1):
    """
    a1 represents the average exec-time of a random query under concurrency
    b1 represents the max level of concurrency in a system
    a2 represents the average impact on a query's runtime when executed concurrently with other queries
    """
    num_concurrency, isolated_runtime = x
    return (a1 * np.maximum(num_concurrency - b1, 0)) + (
        1 + a2 * np.minimum(num_concurrency, b1)
    ) * isolated_runtime


class SimpleFitCurve(ConcurPredictor):
    """
    Simple fit curve model for runtime prediction with concurrency
    runtime = queue_time(num_concurrency) + alpha(num_concurrency) * isolated_runtime
            = (a1 * max(num_concurrency-b1, 0)) + (1 + a2*min(num_concurrency, b1)) * isolated_runtime
    optimize a1, b1, b2
    """

    def __init__(self):
        super().__init__()
        self.isolated_rt_cache = dict()
        self.use_train = True
        self.a1_global = 0
        self.a1 = dict()
        self.b1_global = 0
        self.b1 = dict()
        self.a2_global = 0
        self.a2 = dict()

    def train(self, trace_df, use_train=True, isolated_trace_df=None):
        self.use_train = use_train
        self.get_isolated_runtime_cache(trace_df, isolated_trace_df)
        concurrent_df = trace_df[trace_df["num_concurrent_queries"] > 0]

        global_y = []
        global_x = []
        global_ir = []
        for i, rows in concurrent_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache:
                continue
            isolated_rt = self.isolated_rt_cache[i]
            concurrent_rt = rows["runtime"].values
            if use_train:
                num_concurrency = rows["num_concurrent_queries_train"].values
            else:
                num_concurrency = rows["num_concurrent_queries"].values
            if len(num_concurrency) < 10:
                continue
            global_y.append(concurrent_rt)
            global_x.append(num_concurrency)
            global_ir.append(np.ones(len(num_concurrency)) * isolated_rt)
            fit, _ = optimization.curve_fit(
                simple_queueing_func,
                (num_concurrency, np.ones(len(num_concurrency)) * isolated_rt),
                concurrent_rt,
                np.array([5, 0.1, 20]),
            )
            self.a1[i] = fit[0]
            self.a2[i] = fit[1]
            self.b1[i] = fit[2]
        global_y = np.concatenate(global_y)
        global_x = np.concatenate(global_x)
        global_ir = np.concatenate(global_ir)
        fit, _ = optimization.curve_fit(
            simple_queueing_func,
            (global_x, global_ir),
            global_y,
            np.array([5, 0.1, 20]),
        )
        self.a1_global = fit[0]
        self.a2_global = fit[1]
        self.b1_global = fit[2]

    def predict(self, eval_trace_df, use_global=False):
        predictions = dict()
        labels = dict()
        for i, rows in eval_trace_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache or i not in self.a1:
                continue
            isolated_rt = self.isolated_rt_cache[i]
            label = rows["runtime"].values
            labels[i] = label
            if self.use_train:
                num_concurrency = rows["num_concurrent_queries_train"].values
            else:
                num_concurrency = rows["num_concurrent_queries"].values
            x = (num_concurrency, np.ones(len(num_concurrency)) * isolated_rt)
            if use_global:
                pred = simple_queueing_func(
                    x, self.a1_global, self.a2_global, self.b1_global
                )
            else:
                pred = simple_queueing_func(x, self.a1[i], self.a2[i], self.b1[i])
            pred = np.maximum(pred, 0.001)
            predictions[i] = pred
        return predictions, labels


def interaction_func(
    x, q1, i1, i2, c1, m1, m2, m3, cm1, r1, r2, max_concurrency, avg_io_speed, memory_size
):
    """
    An analytical function that can consider 3 types of resource sharing/contention: IO, memory, CPU
    x:: input tuple containing:
        isolated_runtime: the isolated runtime without concurrency of a query
        avg_runtime: average or median observed runtime of a query under any concurrency
        num_concurrency: number of concurrent queries running with this query
        sum_concurrent_runtime: sum of the estimated runtime of all queries concurrently running with this query (CPU)
        est_scan: estimated MB of data that this query will need to scan (IO)
        est_concurrent_scan: estimated MB of data that the concurrently running queries will need to scan (IO)
        scan_sharing_percentage: estimated percentage of data in cache (sharing) according to concurrent queries
        max_est_card: maximum estimated cardinality in the query plan of this query (reflect peak memory usage)
        avg_est_card: average estimated cardinality in the query plan of this query (reflect average memory usage)
        max_concurrent_card: maximum estimated cardinality for all concurrent queries
        avg_concurrent_card: average estimated cardinality for all concurrent queries
    TODO: adding memory and CPU information
    """
    (
        isolated_runtime,
        avg_runtime,
        num_concurrency,
        sum_concurrent_runtime,
        est_scan,
        est_concurrent_scan,
        scan_sharing_percentage,
        max_est_card,
        avg_est_card,
        max_concurrent_card,
        avg_concurrent_card,
    ) = x
    # fraction of running queries (as opposed to queueing queries)
    running_frac = np.minimum(num_concurrency, max_concurrency) / np.maximum(
        num_concurrency, 1
    )
    # estimate queueing time of a query based on the sum of concurrent queries' run time
    queueing_time = (
        q1
        * (
            np.maximum(num_concurrency - max_concurrency, 0)
            / np.maximum(num_concurrency, 1)
        )
        * sum_concurrent_runtime
    )
    # estimate io_speed of a query assuming each query has a base io_speed of i1 + the io speed due to contention
    io_speed = i1 + avg_io_speed / np.minimum(
        np.maximum(num_concurrency, 1), max_concurrency
    )
    # estimate time speed on IO as the (estimated scan - data in cache) / estimated io_speed
    # use i2 to adjust the estimation error in est_scan and scan_sharing_percentage
    io_time = i2 * est_scan * (1 - scan_sharing_percentage) / io_speed
    # estimate the amount of CPU work/time as the weighted average of isolated_runtime and avg_runtime - io_time
    cpu_time_isolated = (r1 * isolated_runtime + r2 * avg_runtime) - io_time
    # estimate the amount of CPU work imposed by the concurrent queries (approximated by their estimate runtime)
    cpu_concurrent = (running_frac * sum_concurrent_runtime) / avg_runtime
    # estimate the amount of memory imposed by the concurrent queries
    max_mem_usage_perc = max_concurrent_card / (max_concurrent_card + max_est_card)
    avg_mem_usage_perc = avg_concurrent_card / (avg_concurrent_card + avg_est_card)
    memory_concurrent = np.log(
        m1 * np.maximum(max_concurrent_card + max_est_card - memory_size, 0.01) * max_mem_usage_perc
        + m2 * np.maximum(avg_concurrent_card + avg_est_card - memory_size, 0.01) * avg_mem_usage_perc
        + 0.0001
    ) * np.log(m1 * max_est_card + m2 * avg_est_card + 0.0001)
    memory_concurrent = np.maximum(memory_concurrent, 0)
    # estimate the CPU time of a query by considering the contention of CPU and memory of other queries
    cpu_time = (
        1
        + c1 * cpu_concurrent
        + m3 * memory_concurrent
        + cm1 * np.sqrt(cpu_concurrent * memory_concurrent)
    ) * cpu_time_isolated
    # final runtime of a query is estimated to be the queueing time + io_time + cpu_time
    return np.maximum(queueing_time + io_time + cpu_time, 0.01)


class ComplexFitCurve(ConcurPredictor):
    """
    Simple fit curve model for runtime prediction with concurrency
    runtime = queue_time(num_concurrency) + alpha(num_concurrency) * isolated_runtime
            = (a1 * max(num_concurrency-b1, 0)) + (1 + a2*min(num_concurrency, b1)) * isolated_runtime
    optimize a1, b1, b2
    """

    def __init__(self):
        super().__init__()
        self.isolated_rt_cache = dict()
        self.average_rt_cache = dict()
        self.query_info = dict()
        self.db_stats = None
        self.table_sizes = dict()
        self.table_sizes_by_index = dict()
        self.table_nrows_by_index = dict()
        self.table_column_map = dict()
        self.use_train = True
        self.is_column_store = False
        self.q1 = 0.5
        self.i1 = 10
        self.i2 = 2
        self.c1 = 0.2
        self.m1 = 0.5
        self.m2 = 0.5
        self.m3 = 0.1
        self.cm1 = 0.2
        self.r1 = 0.8
        self.r2 = 0.2
        self.max_concurrency = 10
        self.avg_io_speed = 200
        self.memory_size = 16000
        self.bound = optimization.Bounds(
            [0.1, 10, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.5, 0.05, 2, 20, 10000],
            [1, 200, 2, 1, 0.9, 0.9, 0.5, 0.5, 0.95, 0.4, 20, 2000, 50000],
        )

    def _compute_table_size(self):
        for col in self.db_stats["column_stats"]:
            table = col["tablename"]
            if table not in self.table_column_map:
                self.table_sizes[table] = 0
                self.table_column_map[table] = []
            self.table_column_map[table].append(col["attname"])
            if col["avg_width"] is not None and col["avg_width"] > 0:
                self.table_sizes[table] += col["avg_width"]
        all_table_names = [t["relname"] for t in self.db_stats["table_stats"]]
        for table in self.table_sizes:
            if table in all_table_names:
                idx = all_table_names.index(table)
                num_tuples = self.db_stats["table_stats"][idx]["reltuples"]
                self.table_nrows_by_index[idx] = num_tuples
                size_in_mb = (num_tuples * self.table_sizes[table]) / (1024 * 1024)
                self.table_sizes[table] = size_in_mb
                self.table_sizes_by_index[idx] = size_in_mb

    def pre_process_queries(
        self, parsed_queries_path, with_width=True, use_true_card=False
    ):
        plans = load_json(parsed_queries_path, namespace=False)
        self.db_stats = plans["database_stats"]
        self._compute_table_size()
        self.query_info = dict()
        for i in range(len(plans["sql_queries"])):
            curr_query_info = dict()
            curr_query_info["sql"] = plans["sql_queries"][i]
            all_cardinality = []
            dfs_cardinality(
                plans["parsed_plans"][i], all_cardinality, with_width, use_true_card
            )
            curr_query_info["all_cardinality"] = all_cardinality
            est_scan, est_scan_per_table = estimate_scan_in_mb(
                self.db_stats,
                plans["parsed_queries"][i],
                use_true_card,
                self.is_column_store,
            )
            curr_query_info["est_scan"] = est_scan
            curr_query_info["est_scan_per_table"] = est_scan_per_table
            self.query_info[i] = curr_query_info

    def estimate_data_share_percentage(self, idx, concur_info, pre_exec_info=None):
        # TODO: make it smarter by considering buffer pool behavior
        curr_scan = self.query_info[idx]["est_scan_per_table"]
        curr_total_scan = self.query_info[idx]["est_scan"]
        if pre_exec_info is not None:
            concur_info = concur_info + pre_exec_info
        all_shared_scan = 0
        for table in curr_scan:
            table_size = self.table_sizes_by_index[table]
            table_shared_scan = 0
            for c in concur_info:
                concur_scan = self.query_info[c[0]]["est_scan_per_table"]
                if table in concur_scan:
                    concur_scan_perc = concur_scan[table] / table_size
                    overlap_scan = concur_scan_perc * curr_scan[table]
                    table_shared_scan += overlap_scan
            table_shared_scan = min(table_shared_scan, curr_scan[table])
            all_shared_scan += table_shared_scan
        return min(all_shared_scan / curr_total_scan, 1.0)

    def featurize_data(self, concurrent_df):
        global_y = []
        global_isolated_runtime = []
        global_avg_runtime = []
        global_num_concurrency = []
        global_sum_concurrent_runtime = []
        global_est_scan = []
        global_est_concurrent_scan = []
        global_scan_sharing_percentage = []
        global_max_est_card = []
        global_avg_est_card = []
        global_max_concurrent_card = []
        global_avg_concurrent_card = []
        global_query_idx = dict()
        start = 0
        for i, rows in concurrent_df.groupby("query_idx"):
            if (
                i not in self.isolated_rt_cache
                or i not in self.query_info
                or i not in self.average_rt_cache
            ):
                continue
            concurrent_rt = rows["runtime"].values
            query_info = self.query_info[i]
            n_rows = len(rows)
            if self.use_train:
                num_concurrency = rows["num_concurrent_queries_train"].values
                concur_info = rows["concur_info_train"].values
            else:
                num_concurrency = rows["num_concurrent_queries"].values
                concur_info = rows["concur_info"].values
            pre_exec_info = rows["pre_exec_info"].values

            global_query_idx[i] = (start, start + n_rows)
            start += n_rows
            global_y.append(concurrent_rt)
            global_isolated_runtime.append(np.ones(n_rows) * self.isolated_rt_cache[i])
            global_avg_runtime.append(np.ones(n_rows) * self.average_rt_cache[i])
            global_num_concurrency.append(num_concurrency)
            global_est_scan.append(np.ones(n_rows) * query_info["est_scan"])
            global_max_est_card.append(
                np.ones(n_rows) * np.max(query_info["all_cardinality"]) / (1024 * 1024)
            )
            global_avg_est_card.append(
                np.ones(n_rows) * np.average(query_info["all_cardinality"]) / (1024 * 1024)
            )
            for j in range(n_rows):
                sum_concurrent_runtime = 0
                sum_concurrent_scan = 0
                concurrent_card = []
                for c in concur_info[j]:
                    if c[0] in self.average_rt_cache:
                        sum_concurrent_runtime += self.average_rt_cache[c[0]]
                    else:
                        print(c[0])
                    if c[0] in self.query_info:
                        sum_concurrent_scan += self.query_info[c[0]]["est_scan"]
                        concurrent_card.extend(self.query_info[c[0]]["all_cardinality"])
                    else:
                        print(c[0])

                global_sum_concurrent_runtime.append(sum_concurrent_runtime)
                global_est_concurrent_scan.append(sum_concurrent_scan)
                if len(concurrent_card) == 0:
                    global_max_concurrent_card.append(0)
                    global_avg_concurrent_card.append(0)
                else:
                    global_max_concurrent_card.append(np.max(concurrent_card) / (1024 * 1024))
                    global_avg_concurrent_card.append(
                        np.average(concurrent_card) / (1024 * 1024)
                    )
                global_scan_sharing_percentage.append(
                    self.estimate_data_share_percentage(
                        i, concur_info[j], pre_exec_info[j]
                    )
                )

        global_y = np.concatenate(global_y)
        global_isolated_runtime = np.concatenate(global_isolated_runtime)
        global_avg_runtime = np.concatenate(global_avg_runtime)
        global_num_concurrency = np.concatenate(global_num_concurrency)
        global_est_scan = np.concatenate(global_est_scan)
        global_max_est_card = np.concatenate(global_max_est_card)
        global_avg_est_card = np.concatenate(global_avg_est_card)
        global_sum_concurrent_runtime = np.asarray(global_sum_concurrent_runtime)
        global_est_concurrent_scan = np.asarray(global_est_concurrent_scan)
        global_max_concurrent_card = np.asarray(global_max_concurrent_card)
        global_avg_concurrent_card = np.asarray(global_avg_concurrent_card)
        global_scan_sharing_percentage = np.asarray(global_scan_sharing_percentage)
        feature = (
            global_isolated_runtime,
            global_avg_runtime,
            global_num_concurrency,
            global_sum_concurrent_runtime,
            global_est_scan,
            global_est_concurrent_scan,
            global_scan_sharing_percentage,
            global_max_est_card,
            global_avg_est_card,
            global_max_concurrent_card,
            global_avg_concurrent_card,
        )
        return feature, global_y, global_query_idx

    def train(self, trace_df, use_train=True, isolated_trace_df=None):
        self.use_train = use_train
        self.get_isolated_runtime_cache(
            trace_df, isolated_trace_df, get_avg_runtime=True
        )
        concurrent_df = trace_df[trace_df["num_concurrent_queries"] > 0]
        feature, label, _ = self.featurize_data(concurrent_df)

        initial_param_value = np.array(
            [
                self.q1,
                self.i1,
                self.i2,
                self.c1,
                self.m1,
                self.m2,
                self.m3,
                self.cm1,
                self.r1,
                self.r2,
                self.max_concurrency,
                self.avg_io_speed,
                self.memory_size
            ]
        )
        fit, _ = optimization.curve_fit(
            interaction_func,
            feature,
            label,
            initial_param_value,
            bounds=self.bound,
            jac="3-point",
            method="trf",
            loss="soft_l1",
        )
        self.q1 = fit[0]
        self.i1 = fit[1]
        self.i2 = fit[2]
        self.c1 = fit[3]
        self.m1 = fit[4]
        self.m2 = fit[5]
        self.m3 = fit[6]
        self.cm1 = fit[7]
        self.r1 = fit[8]
        self.r2 = fit[9]
        self.max_concurrency = fit[10]
        self.avg_io_speed = fit[11]
        self.memory_size = fit[12]

    def predict(self, eval_trace_df, use_global=False, return_per_query=True):
        feature, labels, query_idx = self.featurize_data(eval_trace_df)
        preds = interaction_func(
            feature,
            self.q1,
            self.i1,
            self.i2,
            self.c1,
            self.m1,
            self.m2,
            self.m3,
            self.cm1,
            self.r1,
            self.r2,
            self.max_concurrency,
            self.avg_io_speed,
            self.memory_size
        )
        if return_per_query:
            preds_per_query = dict()
            labels_per_query = dict()
            for i in query_idx:
                start, end = query_idx[i]
                preds_per_query[i] = preds[start:end]
                labels_per_query[i] = labels[start:end]
            return preds_per_query, labels_per_query
        else:
            return preds, labels


In [101]:
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
cfc = ComplexFitCurve()
cfc.pre_process_queries(parsed_queries_path)
cfc.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df)
predictions_cfc, labels = cfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query)=============")
result_overall_cfc, result_per_query_cfc = cfc.evaluate_performance(concurrency_df.iloc[train_idx], use_global=True)

`ftol` termination condition is satisfied.
Function evaluations 683, initial cost 1.9270e+06, final cost 1.1625e+06, first-order optimality 8.95e+00.
===========Performance for simple linear regression model (all query)=============
50% absolute error is 4.9120826860352995, q-error is 2.160032796688101
90% absolute error is 92.10580902105865, q-error is 7.758735994804825
95% absolute error is 155.30414968965982, q-error is 12.404323857406684


In [162]:
cfc = ComplexFitCurve()
cfc.pre_process_queries(parsed_queries_path)
cfc.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
predictions_cfc, labels = cfc.predict(concurrency_df.iloc[train_idx], use_global=True)
print("===========Performance for simple linear regression model (all query)=============")
result_overall_cfc, result_per_query_cfc = cfc.evaluate_performance(concurrency_df.iloc[train_idx], use_global=True)

===========Performance for simple linear regression model (all query)=============
50% absolute error is 4.608326551755917, q-error is 2.0343009371791063
90% absolute error is 73.38343957667942, q-error is 10.699833830542556
95% absolute error is 116.9916766259407, q-error is 22.27818608283996


In [153]:
predictions_cfc, labels = cfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query)=============")
result_overall_cfc, result_per_query_cfc = cfc.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple linear regression model (all query)=============
50% absolute error is 4.830843702621404, q-error is 2.050948928795532
90% absolute error is 76.4410345261911, q-error is 10.487657930427357
95% absolute error is 121.69564920156964, q-error is 21.74345964592817


In [154]:
cfc.q1, cfc.i1, cfc.i2, cfc.c1, cfc.m1, cfc.m2, cfc.m3, cfc.cm1, cfc.r1, cfc.r2, cfc.max_concurrency, cfc.avg_io_speed, cfc.memory_size

(0.28649072474579773,
 198.5897187207386,
 0.020578385713092135,
 0.010000000000000002,
 0.018182377515533105,
 0.8999999999999999,
 0.010000000000000002,
 0.010000000000000002,
 0.5000000000000001,
 0.33978539225171694,
 5.626611837644638,
 97.69463853931596,
 29151.228451412764)

In [159]:
i = 0
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_cfc[i])
np.stack((predictions_cfc[i][idx], predictions_sfc[i][idx], predictions_xgb[i][idx], labels[i][idx]), axis=1)

29.34782457351685


array([[ 61.65134485,  42.17732367,  17.49212265,  31.13634539],
       [ 64.75384192,  48.56347512,  25.46839142,  11.41822219],
       [ 65.08156396,  61.10030647,  25.34730911,   6.50907421],
       [ 65.23043496,  48.56347512,  13.91628551,  29.3566246 ],
       [ 66.25542233,  61.10030647,  47.251091  ,  12.50359488],
       [ 67.38660402,  73.63713782,  28.58919144,  33.02771139],
       [ 67.44973713,  61.10030647,  43.75050735,  12.576473  ],
       [ 67.57012723,  48.56347512,  17.61630249,  12.10794377],
       [ 71.26073841,  86.17396916,  56.07400513,  46.65917611],
       [ 72.6710367 ,  61.10030647,  16.42707443,  17.26033831],
       [ 78.22775058,  98.71080051,  66.40315247,  39.24779963],
       [ 92.69759053,  98.71080051, 136.74949646, 182.27577138],
       [ 96.50326528, 211.54228264, 143.23944092, 130.59745121],
       [108.57401209, 111.24763186,  88.04031372, 131.94944143],
       [114.85303304, 111.24763186,  46.63616562, 108.15937161],
       [116.82237385, 136

In [ ]:
plan = load_json(parsed_queries_path, namespace=False)

In [ ]:
plan[""]

In [ ]:
plan["parsed_queries"][-1]

In [ ]:
plan.keys()

In [ ]:
np.maximum(np.zeros(3), 1)

In [156]:
sfc = SimpleFitCurve()
sfc.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
#predictions, labels = sfc.predict(eval_trace_df)
#print("===========Performance for simple curve fitting model (per query)=============")
#result_overall, result_per_query = sfc.evaluate_performance(eval_trace_df, use_train=True)
predictions_sfc, labels = sfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple curve fitting model (all query)=============")
result_overall_sfc, result_per_query_sfc = sfc.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple curve fitting model (all query)=============
50% absolute error is 9.627042932918474, q-error is 2.4357127503878613
90% absolute error is 75.47100542916611, q-error is 15.68507968881842
95% absolute error is 123.25611489299501, q-error is 38.380486692719245


/Users/ziniuw/.local/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [157]:
xgb = XGBoostPredictor(k=240)
xgb.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df, use_pre_exec_info=True)
predictions_xgb, labels = xgb.predict(eval_trace_df)
#predictions_xgb, labels = xgb.predict(concurrency_df.iloc[train_idx], use_train=False)
print("===========Performance for XGBoost model (train on full)=============")
result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(eval_trace_df)
#result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(concurrency_df.iloc[train_idx], use_train=False)

===========Performance for XGBoost model (train on full)=============
50% absolute error is 9.802342653274536, q-error is 2.5974413022067666
90% absolute error is 63.95580263137816, q-error is 495.0843460841116
95% absolute error is 99.69837267398795, q-error is 1601.4952852611802


In [160]:
np.log(0.001)

-6.907755278982137